Prototype of the `video_reconstruction.py` script in the `scripts` folder.  
Does Sparse and Dense Reconstruction for every frame of Marc's dataset. Several types of optimization are supported by changing the `reuse_previous_frame` and `keyframes` variables below

In [ ]:
%cd ..
%reload_ext autoreload
%autoreload 2

In [ ]:
import pyrender
import matplotlib.pyplot as plt

from face_reconstruction.pipeline import BFMPreprocessor
from face_reconstruction.plots import PlotManager, plot_params
from face_reconstruction.optim import NearestNeighborMode, DistanceType, run_icp, BFMOptimization, KeyframeOptimizationParameters, run_icp_keyframes, BFMOptimizationParameters

# 1. Preprocessing

In [ ]:
preprocessor = BFMPreprocessor()

In [ ]:
def preprocess(frame_id):
    img, depth_img, intrinsics = preprocessor.load_frame(frame_id)
    pointcloud, pointcloud_normals, colors = preprocessor.to_3d(img, depth_img, intrinsics)
    landmark_points, bfm_landmark_indices, face_pointcloud, face_pointcloud_colors = preprocessor.detect_landmarks(img, depth_img, intrinsics)
    return bfm_landmark_indices, landmark_points, face_pointcloud, pointcloud_normals

# 2. Sparse Reconstruction

In [ ]:
reuse_previous_frame = True
keyframes = [0, 25]

In [ ]:
n_params_shape_sparse = 3
n_params_expression_sparse = 0
weight_shape_params_sparse = 100
weight_expression_params_sparse = 1
l2_regularization_sparse = 10000

In [ ]:
nn_mode = NearestNeighborMode.FACE_VERTICES # FACE_VERTICES: every face vertex will be assigned its nearest neighbor in pointcloud
                                            # POINTCLOUD: every point in pointcloud will be assigned its nearest neighbor in face model
distance_type = DistanceType.POINT_TO_POINT
icp_iterations = 2
optimization_steps_per_iteration = 10
l2_regularization_dense = 4000 # 10000 for Lie

n_params_shape_dense = 30 # 20
n_params_expression_dense = 30 # 10
weight_shape_params_dense = 100 # 10000, 10000000000 for POINT_TO_PLANE
weight_expression_params_dense = 100 # 1000, 10000000000 for POINT_TO_PLANE

In [ ]:
sparse_optimizer = BFMOptimization(preprocessor.bfm, 
                               n_params_shape=n_params_shape_sparse,
                               n_params_expression=n_params_expression_sparse, 
                               weight_shape_params=weight_shape_params_sparse, 
                               weight_expression_params=weight_expression_params_sparse,
                               rotation_mode='lie')

dense_optimizer = BFMOptimization(preprocessor.bfm, 
                           n_params_shape=n_params_shape_dense,
                           n_params_expression=n_params_expression_dense, 
                           weight_shape_params=weight_shape_params_dense, 
                           weight_expression_params=weight_expression_params_dense,
                           rotation_mode='lie')

In [ ]:
def run_sparse_optimization(sparse_optimizer, bfm_landmark_indices, landmark_points, initial_params):
    sparse_loss = sparse_optimizer.create_sparse_loss_3d(bfm_landmark_indices, landmark_points, regularization_strength=l2_regularization_sparse)
    sparse_context = sparse_optimizer.create_optimization_context(sparse_loss, initial_params)
    result = sparse_context.run_optimization(sparse_loss)
    return sparse_context.create_parameters_from_theta(result.x)

In [ ]:
def run_dense_optimization(dense_optimizer, face_pointcloud, pointcloud_normals, params_sparse):
    params, distances, dense_param_history = run_icp(dense_optimizer, 
                               face_pointcloud,
                               preprocessor.bfm, 
                               params_sparse.with_new_manager(dense_optimizer), 
                               max_iterations=icp_iterations, 
                               nearest_neighbor_mode=nn_mode, 
                               distance_type=distance_type,
                               max_nfev=optimization_steps_per_iteration,
                               l2_regularization=l2_regularization_dense,
                              pointcloud_normals=pointcloud_normals)
    return params, dense_param_history

In [ ]:
plot_manager = PlotManager.new_run("video_reconstruction")

In [ ]:
initial_params = preprocessor.get_initial_params(sparse_optimizer)

In [ ]:
if keyframes:
    print(f"===== Estimating shape coefficients from keyframes =====")
    
    # Only optimize for shape coefficients
    sparse_optimizer = BFMOptimization(preprocessor.bfm, 
                               n_params_shape=n_params_shape_sparse,
                               n_params_expression=0, 
                               weight_shape_params=weight_shape_params_sparse, 
                               weight_expression_params=weight_expression_params_sparse,
                               rotation_mode='lie')

    dense_optimizer = BFMOptimization(preprocessor.bfm, 
                               n_params_shape=n_params_shape_dense,
                               n_params_expression=0, 
                               weight_shape_params=weight_shape_params_dense, 
                               weight_expression_params=weight_expression_params_dense,
                               rotation_mode='lie')
    
    img_landmark_points = []
    pointclouds = []
    pointcloud_normals_list = []
    bfm_landmark_indices_list = []
    for frame_id in keyframes:
        bfm_landmark_indices, landmark_points, face_pointcloud, pointcloud_normals = preprocess(frame_id)
        bfm_landmark_indices_list.append(bfm_landmark_indices)
        img_landmark_points.append(landmark_points)
        pointclouds.append(face_pointcloud)
        pointcloud_normals_list.append(pointcloud_normals)
        
    initial_params_keyframe = KeyframeOptimizationParameters(sparse_optimizer, [0 for _ in range(n_params_shape_sparse)], [initial_params.camera_pose for _ in range(len(keyframes))])
    sparse_keyframe_loss = sparse_optimizer.create_sparse_keyframe_loss(bfm_landmark_indices_list, img_landmark_points, regularization_strength = l2_regularization_sparse)
    sparse_context = sparse_optimizer.create_optimization_context(sparse_keyframe_loss, initial_params_keyframe)
    result = sparse_context.run_optimization()
    
    initial_params_keyframe_dense = KeyframeOptimizationParameters.from_theta(sparse_context, result.x).with_new_manager(dense_optimizer)
    params_dense, distances, dense_param_history = run_icp_keyframes(dense_optimizer, 
                               pointclouds,
                               preprocessor.bfm, 
                               initial_params_keyframe_dense, 
                               max_iterations=icp_iterations, 
                               nearest_neighbor_mode=nn_mode, 
                               distance_type=distance_type,
                               max_nfev=optimization_steps_per_iteration,
                               l2_regularization=l2_regularization_dense,
                              pointcloud_normals=pointcloud_normals_list)
    
    # Don't optimize for shape coefficients anymore
    sparse_optimizer = BFMOptimization(preprocessor.bfm, 
                               n_params_shape=0,
                               n_params_expression=n_params_expression_sparse, 
                               weight_shape_params=weight_shape_params_sparse, 
                               weight_expression_params=weight_expression_params_sparse,
                               rotation_mode='lie')

    dense_optimizer = BFMOptimization(preprocessor.bfm, 
                               n_params_shape=0,
                               n_params_expression=n_params_expression_dense, 
                               weight_shape_params=weight_shape_params_dense, 
                               weight_expression_params=weight_expression_params_dense,
                               rotation_mode='lie')
    
    initial_params = BFMOptimizationParameters(sparse_optimizer, params_dense.shape_coefficients, initial_params.expression_coefficients, initial_params.camera_pose)
    plot_manager.save_params(initial_params, "keyframe_params")

In [ ]:
for frame_id in preprocessor.loader.get_frame_ids():
    print(f"===== Frame {frame_id} ======")
    bfm_landmark_indices, landmark_points, face_pointcloud, pointcloud_normals = preprocess(frame_id)
    print(f"  --- Sparse Reconstruction ---")
    params_sparse = run_sparse_optimization(sparse_optimizer, bfm_landmark_indices, landmark_points, initial_params)
    print(f"  --- Dense Reconstruction ---")
    params_dense, param_history = run_dense_optimization(dense_optimizer, face_pointcloud, pointcloud_normals, params_sparse)
    
    img_with_mask = preprocessor.render_onto_img(params_dense)
    plot_manager.save_image(img_with_mask, f"frame_{frame_id:04d}.jpg")
    plot_manager.save_params(params_dense, f"params_{frame_id:04d}")
    plot_manager.save_param_history(param_history, f"param_history_{frame_id:04d}")
    
    if reuse_previous_frame:
        initial_params = params_dense.with_new_manager(sparse_optimizer)
    print()
    print()

# 4. Rendering

In [ ]:
frame_id = 2

## 4.1. Interactive Rendering

In [ ]:
params = plot_manager.load_params(f"params_{frame_id:04d}", preprocessor.bfm)

In [ ]:
scene = preprocessor.setup_scene(initial_params)

In [ ]:
pyrender.Viewer(scene, use_raymond_lighting=True, viewport_size=(preprocessor.img_width, preprocessor.img_height))

## 4.2. 2D Rendering

In [ ]:
img_with_mask = preprocessor.render_onto_img(initial_params)
plt.imshow(img_with_mask)
plt.show()